# Notities

## Handige links
* [Statline portaal](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [Snelstartgids open data](https://www.cbs.nl/nl-nl/onze-diensten/open-data/statline-als-open-data/snelstartgids)
* [repo Github voor cbsodata](https://github.com/J535D165/cbsodata)
* [PyPi documentatis cbsodata](https://pypi.org/project/cbsodata/)
* [Sphinx docs cbsodata](https://cbsodata.readthedocs.io/en/latest/index.html)
* [GitHub voorbeelden OData 3](https://github.com/statistiekcbs/CBS-Open-Data-v3)
* [GitHub voorbeelden OData 4](https://github.com/statistiekcbs/CBS-Open-Data-v4/tree/master/Python)
* [Statline voor derden](https://www.cbs.nl/nl-nl/cijfers/statline/statline-voor-derden)
* [CBS open data Statline](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [PDOK Viewer: Kies bij dataset voor CBS gebiedsindelingen (wijken staan helemaal onderaan) voor kaartweergave](https://www.pdok.nl/viewer/)

# Imports

In [1]:
import pandas as pd
import cbsodata

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import altair as alt
from sklearn import preprocessing

# Settings

In [2]:
GEMEENTES_REGIO_NIJMEGEN = ['Berg en Dal', 'Beuningen', 'Druten', 'Heumen', 'Mook en Middelaar', 'Nijmegen', 'Wijchen']

KEYWORDS = ['wmo', 'gemeente', 'regio', 'postcode', 'zorg', 'werk', 'lonen', 'loon', 'inkomen', 'conjunctuur', 
            'prognose', 'afstand', 'wijk', 'voorziening', 'genees', 'gezond']
URLS = ['opendata.cbs.nl', 'dataderden.cbs.nl']

CBS_OPEN_URL = 'opendata.cbs.nl'
WMO_TABLES = {'2020': '84907NED',
            '2019': '84664NED',
            '2018': '84421NED',
            '2017': '83818NED',
            '2016': '83620NED',
            '2015': '83267NED'}
WIJK_TABLES = {'2020': '84799NED', 
               '2019': '84583NED', 
               '2018': '84286NED', 
               '2017': '83765NED', 
               '2016': '83487NED', 
               '2015': '83220NED'} 
#                '2014': '82931NED'}

DICT_REALTIVELY_COLS = {'aantalinwoners': ['percentagewmoclienten', 'mannen', 'vrouwen', 'k0tot15jaar', 'k15tot25jaar',
                                           'k25tot45jaar', 'k45tot65jaar', 'k65jaarofouder', 'ongehuwd', 'gehuwd', 'gescheiden',
                                           'verweduwd', 'westerstotaal', 'nietwesterstotaal', 'marokko', 
                                           'nederlandseantillenenaruba', 'suriname', 'turkije', 'overignietwesters', 
                                           'geboortetotaal', 'geboorterelatief', 'sterftetotaal', 'sterfterelatief', 
                                           'aantalinkomensontvangers', 'personenpersoortuitkeringbijstand', 
                                           'personenpersoortuitkeringao', 'personenpersoortuitkeringww', 
                                           'personenpersoortuitkeringaow'],
                        'huishoudenstotaal': ['eenpersoonshuishoudens', 'huishoudenszonderkinderen', 'huishoudensmetkinderen'],
                        'bedrijfsvestigingentotaal': ['alandbouwbosbouwenvisserij', 'bfnijverheidenenergie', 
                                                      'gihandelenhoreca', 'hjvervoerinformatieencommunicatie', 
                                                      'klfinancieledienstenonroerendgoed', 'mnzakelijkedienstverlening', 
                                                      'rucultuurrecreatieoverigediensten'],
                        'personenautostotaal': ['personenautosbrandstofbenzine', 'personenautosoverigebrandstof'],
                        'oppervlaktetotaal': ['oppervlakteland', 'oppervlaktewater']}
LIST_NORM_COLS = ['wmoclienten', 'aantalinwoners', 'huishoudenstotaal', 'gemiddeldehuishoudensgrootte', 'bevolkingsdichtheid', 
                  'woningvoorraad', 'gemiddeldewoningwaarde', 'percentageeengezinswoning', 'percentagemeergezinswoning', 
                  'percentagebewoond', 'percentageonbewoond', 'koopwoningen', 'huurwoningentotaal', 'inbezitwoningcorporatie', 
                  'inbezitoverigeverhuurders', 'eigendomonbekend', 'bouwjaarvoor2000', 'bouwjaarvanaf2000', 
                  'gemiddeldelektriciteitsverbruiktotaal', 'gemelectriciteitsverbruikappartement', 
                  'gemelectriciteitsverbruiktussenwoning', 'gemelectriciteitsverbruikhoekwoning', 
                  'gemelectriciteitsverbruiktweeondereenkapwoning', 'gemelectriciteitsverbruikvrijstaandewoning', 
                  'gemelectriciteitsverbruikhuurwoning', 'gemelectriciteitsverbruikeigenwoning', 
                  'gemiddeldaardgasverbruiktotaal', 'gemgasverbruikappartement', 'gemgasverbruiktussenwoning', 
                  'gemgasverbruikhoekwoning', 'gemgasverbruiktweeondereenkapwoning', 'gemgasverbruikvrijstaandewoning', 
                  'gemgasverbruikhuurwoning', 'gemgasverbruikeigenwoning', 'percentagewoningenmetstadsverwarming',
                  'gemiddeldinkomenperinkomensontvanger', 'gemiddeldinkomenperinwoner', 'k40personenmetlaagsteinkomen', 
                  'k20personenmethoogsteinkomen', 'actieven1575jaar', 'k40huishoudensmetlaagsteinkomen', 
                  'k20huishoudensmethoogsteinkomen', 'huishoudensmeteenlaaginkomen', 'huishonderofrondsociaalminimum',
                  'bedrijfsvestigingentotaal', 'personenautostotaal', 'motorfietsen', 'afstandtothuisartsenpraktijk', 
                  'afstandtotgrotesupermarkt', 'afstandtotkinderdagverblijf', 'afstandtotschool', 'scholenbinnen3km',
                  'oppervlaktetotaal', 'matevanstedelijkheid', 'omgevingsadressendichtheid']
DROP_COLS = ['financieringsvorm', 'wmoclientenper1000inwoners', 'gemeentenaam', 'meestvoorkomendepostcode', 'dekkingspercentage', 'totaaldiefstaluitwoningschuured', 
               'vernielingmisdrijftegenopenbareorde', 'geweldsenseksuelemisdrijven', 'personenautosjongerdan6jaar', 
               'personenautos6jaarenouder', 'bedrijfsmotorvoertuigen']

# Functies

In [3]:
def make_cols_relatively(df, dict_relatively_cols):
    for base_col, relatively_cols in dict_relatively_cols.items():
        df[relatively_cols] = df[relatively_cols].div(df[base_col], axis=0)
    return df

def normalize_cols(df, norm_cols):
    df_sub = df.copy()
    df_sub = df_sub[norm_cols]
    x = df_sub.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_sub=pd.DataFrame(x_scaled, columns=df_sub.columns, index=df_sub.index)
    df = df.drop(norm_cols, axis=1).join(df_sub)
    return df

#  Load dataset

In [4]:
df_dataset_WMO = pd.read_parquet('../data/df_dataset_WMO.parquet.gzip')
print(f"Shape of dataset: {df_dataset_WMO.shape}")
df_dataset_WMO.sample(5)

Shape of dataset: (11267, 109)


,,financieringsvorm,wmoclienten,wmoclientenper1000inwoners,gemeentenaam,aantalinwoners,mannen,vrouwen,k0tot15jaar,k15tot25jaar,k25tot45jaar,k45tot65jaar,k65jaarofouder,ongehuwd,gehuwd,gescheiden,verweduwd,westerstotaal,nietwesterstotaal,marokko,nederlandseantillenenaruba,suriname,turkije,overignietwesters,geboortetotaal,geboorterelatief,sterftetotaal,sterfterelatief,huishoudenstotaal,eenpersoonshuishoudens,huishoudenszonderkinderen,huishoudensmetkinderen,gemiddeldehuishoudensgrootte,bevolkingsdichtheid,woningvoorraad,gemiddeldewoningwaarde,percentageeengezinswoning,percentagemeergezinswoning,percentagebewoond,percentageonbewoond,koopwoningen,huurwoningentotaal,inbezitwoningcorporatie,inbezitoverigeverhuurders,eigendomonbekend,bouwjaarvoor2000,bouwjaarvanaf2000,gemiddeldelektriciteitsverbruiktotaal,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruikeigenwoning,gemiddeldaardgasverbruiktotaal,gemgasverbruikappartement,gemgasverbruiktussenwoning,gemgasverbruikhoekwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgasverbruikhuurwoning,gemgasverbruikeigenwoning,percentagewoningenmetstadsverwarming,aantalinkomensontvangers,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,k40personenmetlaagsteinkomen,k20personenmethoogsteinkomen,actieven1575jaar,k40huishoudensmetlaagsteinkomen,k20huishoudensmethoogsteinkomen,huishoudensmeteenlaaginkomen,huishonderofrondsociaalminimum,personenpersoortuitkeringbijstand,personenpersoortuitkeringao,personenpersoortuitkeringww,personenpersoortuitkeringaow,bedrijfsvestigingentotaal,alandbouwbosbouwenvisserij,bfnijverheidenenergie,gihandelenhoreca,hjvervoerinformatieencommunicatie,klfinancieledienstenonroerendgoed,mnzakelijkedienstverlening,rucultuurrecreatieoverigediensten,personenautostotaal,personenautosbrandstofbenzine,personenautosoverigebrandstof,personenautosperhuishouden,personenautosnaaroppervlakte,motorfietsen,afstandtothuisartsenpraktijk,afstandtotgrotesupermarkt,afstandtotkinderdagverblijf,afstandtotschool,scholenbinnen3km,oppervlaktetotaal,oppervlakteland,oppervlaktewater,meestvoorkomendepostcode,dekkingspercentage,matevanstedelijkheid,omgevingsadressendichtheid,totaaldiefstaluitwoningschuured,vernielingmisdrijftegenopenbareorde,geweldsenseksuelemisdrijven,personenautosjongerdan6jaar,personenautos6jaarenouder,bedrijfsmotorvoertuigen
codering_regio,perioden,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
WK003401,2015,Totaal,1490.0,70.0,Almere,21935,10575,11360,3685,2430,5335,6790,3690,10075,8465,2410,985,2270,4010,740,455,1320,175,1315,235,10,155,7,10220,3730,2950,3535,2.1,1053.0,10127,179.0,67.0,33.0,97.0,3.0,54.0,46.0,40.0,6.0,0.0,93.0,7.0,3040.0,2210.0,3260.0,3320.0,4160.0,5100.0,2470.0,3520.0,1210.0,950.0,1230.0,1380.0,1490.0,2350.0,1100.0,1330.0,NaN,17200.0,29.7,23.5,40.3,17.6,53.1,44.1,16.6,12.3,10.2,1000.0,1380.0,700.0,3640.0,1540.0,5.0,240.0,315.0,200.0,120.0,445.0,215.0,8950,7280,1675,0.9,429.0,845,0.9,0.9,0.7,0.7,8.0,2142,2084,58,.,NaN,3.0,1388.0,NaN,NaN,NaN,3150.0,5805.0,635.0
WK189400,2015,Totaal,140.0,45.0,Peel en Maas,2985,1580,1400,465,395,655,990,490,1405,1350,125,105,185,25,5,5,5,5,10,15,5,15,6,1075,220,385,475,2.6,128.0,1094,238.0,95.0,5.0,97.0,3.0,76.0,24.0,16.0,7.0,1.0,88.0,12.0,4150.0,2980.0,3110.0,3200.0,3910.0,4870.0,2830.0,4560.0,1800.0,1220.0,1200.0,1350.0,1560.0,2200.0,1340.0,1900.0,NaN,2300.0,25.4,21.1,43.2,14.1,62.6,27.8,24.2,2.9,2.6,10.0,250.0,100.0,480.0,255.0,85.0,35.0,35.0,10.0,20.0,35.0,35.0,1555,1230,325,1.4,66.0,205,4.8,4.8,0.8,0.8,1.1,2342,2339,2,.,NaN,5.0,112.0,NaN,NaN,NaN,220.0,1335.0,295.0
WK039605,2018,Totaal,480.0,96.0,Heemskerk,5020,2355,2665,800,550,1185,1315,1180,2280,1810,615,325,575,1050,115,25,80,330,495,45,10,70,15,2405,995,640,780,2.1,743

In [5]:
df_dataset_WMO.shape

(11267, 109)

## Ways to get a subset of the DataFrame:

In [6]:
# Subset on columnvalue:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

# One row / record
df_dataset_WMO.loc[('WK026801', '2019')]

# Multiple rows / records based on combination of the multiindex
df_dataset_WMO.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# Multiple rows / records for one column (works only for series)
df_dataset_WMO['gemeentenaam'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# Multiple rows based on both indexes:
df_dataset_WMO.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# Subset with IndexSlice
idx = pd.IndexSlice
# Subset on one of the multiindex and select a column
df_dataset_WMO.loc[idx[:, ['2018', '2019']], idx["gemeentenaam"]]
# Subset on one of the multiindex and select all columns
df_dataset_WMO.loc[idx['WK026801', :], idx[:]]

financieringsvorm  wmoclienten  wmoclientenper1000inwoners gemeentenaam  aantalinwoners  mannen  vrouwen  k0tot15jaar  k15tot25jaar  k25tot45jaar  k45tot65jaar  k65jaarofouder  ongehuwd  gehuwd  gescheiden  verweduwd  westerstotaal  nietwesterstotaal  marokko  nederlandseantillenenaruba  suriname  turkije  overignietwesters  geboortetotaal  geboorterelatief  sterftetotaal  sterfterelatief  huishoudenstotaal  eenpersoonshuishoudens  huishoudenszonderkinderen  huishoudensmetkinderen  gemiddeldehuishoudensgrootte  bevolkingsdichtheid  woningvoorraad  gemiddeldewoningwaarde  percentageeengezinswoning  percentagemeergezinswoning  percentagebewoond  percentageonbewoond  koopwoningen  huurwoningentotaal  inbezitwoningcorporatie  inbezitoverigeverhuurders  eigendomonbekend  bouwjaarvoor2000  bouwjaarvanaf2000  gemiddeldelektriciteitsverbruiktotaal  gemelectriciteitsverbruikappartement  gemelectriciteitsverbruiktussenwoning  gemelectriciteitsverbruikhoekwoning  \
codering_regio perioden                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
WK026801       2019                Totaal        610.0                        51.0     Nijmegen           11850    5865     5985          320          3915          4610          1850            1165      9510    1305         790        240           1755               1075       90                         110        75      125                685              50                 4             40                3               9020                    6915                       1650                     450                           1.3               9523.0            5480                   222.0                       12.0                        88.0               94.0                  6.0          23.0                76.0                     31.0                       45.0               1.0              84.0               16.0                                 2070.0                                1950.0                                 2860.0                               2810.0   
               2018                Totaal        585.0                        49.0     Nijmegen           12055    5925     6130          440          4090          4510          1895            1135      9640    1370         800        235           1670               1245      100                         110        75      125                845              40                 3             40                3               9060                    6980                       1620                     450                           1.3               9685.0            5427                   198.0                       12.0                        88.0               93.0                  7.0          22.0                76.0                     32.0                       43.0               2.0              84.0               16.0                                 2110.0                                2000.0                                 2840.0                               2910.0   
               2017                Totaal        550

## Transform data

In [7]:
Xy = df_dataset_WMO.copy()
Xy['percentagewmoclienten'] = Xy['wmoclienten']
Xy = Xy.drop(DROP_COLS, axis=1)
Xy = make_cols_relatively(df=Xy, dict_relatively_cols=DICT_REALTIVELY_COLS)
Xy = normalize_cols(df=Xy, norm_cols=LIST_NORM_COLS)

In [8]:
Xy.shape

(11267, 99)

# Challenges data understanding

* Fix missing values
* Define Y and X
* Train - test split
* Use simple model to train and predict
* Try to find a metric to evaluate
* Try to find a way to evaluate feature importance
* When time is left: 
    * Try different models
    * Try classification model (>=0.05% = 1, <0.05% =0... OR: >=mean nation = 1, < mean =0)

In [9]:
Xy

,,mannen,vrouwen,k0tot15jaar,k15tot25jaar,k25tot45jaar,k45tot65jaar,k65jaarofouder,ongehuwd,gehuwd,gescheiden,verweduwd,westerstotaal,nietwesterstotaal,marokko,nederlandseantillenenaruba,suriname,turkije,overignietwesters,geboortetotaal,geboorterelatief,sterftetotaal,sterfterelatief,eenpersoonshuishoudens,huishoudenszonderkinderen,huishoudensmetkinderen,aantalinkomensontvangers,personenpersoortuitkeringbijstand,personenpersoortuitkeringao,personenpersoortuitkeringww,personenpersoortuitkeringaow,alandbouwbosbouwenvisserij,bfnijverheidenenergie,gihandelenhoreca,hjvervoerinformatieencommunicatie,klfinancieledienstenonroerendgoed,mnzakelijkedienstverlening,rucultuurrecreatieoverigediensten,personenautosbrandstofbenzine,personenautosoverigebrandstof,personenautosperhuishouden,personenautosnaaroppervlakte,oppervlakteland,oppervlaktewater,percentagewmoclienten,wmoclienten,aantalinwoners,huishoudenstotaal,gemiddeldehuishoudensgrootte,bevolkingsdichtheid,woningvoorraad,gemiddeldewoningwaarde,percentageeengezinswoning,percentagemeergezinswoning,percentagebewoond,percentageonbewoond,koopwoningen,huurwoningentotaal,inbezitwoningcorporatie,inbezitoverigeverhuurders,eigendomonbekend,bouwjaarvoor2000,bouwjaarvanaf2000,gemiddeldelektriciteitsverbruiktotaal,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruikeigenwoning,gemiddeldaardgasverbruiktotaal,gemgasverbruikappartement,gemgasverbruiktussenwoning,gemgasverbruikhoekwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgasverbruikhuurwoning,gemgasverbruikeigenwoning,percentagewoningenmetstadsverwarming,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,k40personenmetlaagsteinkomen,k20personenmethoogsteinkomen,actieven1575jaar,k40huishoudensmetlaagsteinkomen,k20huishoudensmethoogsteinkomen,huishoudensmeteenlaaginkomen,huishonderofrondsociaalminimum,bedrijfsvestigingentotaal,personenautostotaal,motorfietsen,afstandtothuisartsenpraktijk,afstandtotgrotesupermarkt,afstandtotkinderdagverblijf,afstandtotschool,scholenbinnen3km,oppervlaktetotaal,matevanstedelijkheid,omgevingsadressendichtheid
codering_regio,perioden,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
WK168000,2019,0.488178,0.513213,0.158554,0.095967,0.173853,0.319889,0.250348,0.410292,0.457580,0.065369,0.063978,0.036161,0.020862,0.000000,0.001391,0.001391,0.000000,0.019471,0.005563,0.001391,0.008345,0.002503,0.276527,0.376206,0.347267,NaN,0.011127,0.027816,0.016690,0.230876,0.068966,0.155172,0.224138,0.034483,0.103448,0.275862,0.137931,0.786967,0.213033,1.3,136.0,0.991881,0.008796,0.059805,0.024063,0.032903,0.033753,0.419355,0.008611,0.035345,0.165750,0.91,0.09,0.959184,0.040816,0.74,0.26,0.20,0.06,0.000000,0.88,0.12,0.326055,0.258013,0.236287,0.283784,0.278523,0.239316,0.238361,0.296403,0.331959,0.254902,0.413462,0.440367,0.358974,0.293860,0.273885,0.373418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030769,0.028124,0.052055,0.068376,0.068966,0.017699,0.06,0.042763,0.058379,1.00,0.032325
WK168001,2019,0.510563,0.492958,0.140845,0.105634,0.193662,0.345070,0.211268,0.426056,0.454225,0.066901,0.049296,0.038732,0.014085,0.000000,0.000000,0.003521,0.000000,0.010563,0.007042,0.004930,0.007042,0.005634,0.296000,0.384000,0.328000,NaN,0.014085,0.035211,0.014085,0.197183,0.148148,0.222222,0.185185,0.074074,0.111111,0.148148,0.148148,0.754286,0.245714,1.4,49.0,0.996087,0.003913,0.038732,0.006156,0.012997,0.013566,0.419355,0.002788,0.012781,0.168107,0.97,0.03,0.959184,0.040816,0.83,0.17,0.13,0.04,0.000000,0.95,0.05,0.343523,0.307692,0.147679,0.204955,0.253356,0.222222,0.201117,0.299281,0.367010,0.405229,0.378205,0.388379,0.351981,0.302632,0.299363,0.394515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014324,0.012335,0.030137,0.316239,0.068966,0.053097,0.08,0

In [10]:
# functie om het aantal lege rijen (empty rows) en percentage lege rijen te berekenen.
# let op: laat alleen kolommen zien met lege waarden (door de nulls != 0)
def df_er(df):
    """Calculates nulls and percentage null per column """
    nulls = df.isnull().sum()
    nulls_percentage = df.isnull().sum() * 100 / len(df)
    return pd.concat([nulls[nulls != 0], nulls_percentage[nulls_percentage != 0]], axis=1, keys=['Number of null values', 'Percentage of null values']).sort_values('Number of null values',ascending=False)
df_er(Xy)

,Number of null values,Percentage of null values
percentagewoningenmetstadsverwarming,10614,94.204313
gemiddeldinkomenperinkomensontvanger,4383,38.901216
gemiddeldinkomenperinwoner,4219,37.445638
huishonderofrondsociaalminimum,3493,31.002041
huishoudensmeteenlaaginkomen,3493,31.002041
k20huishoudensmethoogsteinkomen,3489,30.966539
k40huishoudensmetlaagsteinkomen,3489,30.966539
actieven1575jaar,3247,28.818674
k40personenmetlaagsteinkomen,3238,28.738795
k20personenmethoogsteinkomen,3238,28.738795
